In [1]:
import pandas as pd
from  sqlalchemy import create_engine

# Extracting CSVs into DataFrames

In [2]:
income = pd.read_csv("Resources/income.csv", encoding='ISO-8859-1')
income.head()

,id,State_Code,State_Name,State_ab,County,City,Place,Type,Primary,Zip_Code,Area_Code,ALand,AWater,Lat,Lon,Mean,Median,Stdev,sum_w
0,1011000,1,Alabama,AL,Mobile County,Chickasaw,Chickasaw city,City,place,36611,251,10894952,909156,30.771450,-88.079697,38773,30506,33101,1638.260513
1,1011010,1,Alabama,AL,Barbour County,Louisville,Clio city,City,place,36048,334,26070325,23254,31.708516,-85.611039,37725,19528,43789,258.017685
2,1011020,1,Alabama,AL,Shelby County,Columbiana,Columbiana city,City,place,35051,205,44835274,261034,33.191452,-86.615618,54606,31930,57348,926.031000
3,1011030,1,Alabama,AL,Mobile County,Satsuma,Creola city,City,place,36572,251,36878729,2374530,30.874343,-88.009442,63919,52814,47707,378.114619
4,1011040,1,Alabama,AL,Mobile County,Dauphin Island,Dauphin Island,Town,place,36528,251,16204185,413605152,30.250913,-88.171268,77948,67225,54270,282.320328


In [3]:
fast_food = pd.read_csv("Resources/Fast_Food_Restaurants.csv")
fast_food.head()

,id,dateAdded,dateUpdated,address,categories,city,country,keys,latitude,longitude,name,postalCode,province,sourceURLs,websites
0,AVwcmSyZIN2L1WUfmxyw,2015-10-19T23:47:58Z,2018-06-26T03:00:14Z,800 N Canal Blvd,American Restaurant and Fast Food Restaurant,Thibodaux,US,us/la/thibodaux/800ncanalblvd/1780593795,29.814697,-90.814742,SONIC Drive In,70301.0,LA,https://foursquare.com/v/sonic-drive-in/4b7361...,https://locations.sonicdrivein.com/la/thibodau...
1,AVwcmSyZIN2L1WUfmxyw,2015-10-19T23:47:58Z,2018-06-26T03:00:14Z,800 N Canal Blvd,Fast Food Restaurants,Thibodaux,US,us/la/thibodaux/800ncanalblvd/1780593795,29.814697,-90.814742,SONIC Drive In,70301.0,LA,https://foursquare.com/v/sonic-drive-in/4b7361...,https://locations.sonicdrivein.com/la/thibodau...
2,AVwcopQoByjofQCxgfVa,2016-03-29T05:06:36Z,2018-06-26T02:59:52Z,206 Wears Valley Rd,Fast Food Restaurant,Pigeon Forge,US,us/tn/pigeonforge/206wearsvalleyrd/-864103396,35.803788,-83.580553,Taco Bell,37863.0,TN,https://www.yellowpages.com/pigeon-forge-tn/mi...,"http://www.tacobell.com,https://locations.taco..."
3,AVweXN5RByjofQCxxilK,2017-01-03T07:46:11Z,2018-06-26T02:59:51Z,3652 Parkway,Fast Food,Pigeon Forge,US,us/tn/pigeonforge/3652parkway/93075755,35.782339,-83.551408,Arby's,37863.0,TN,http://www.yellowbook.com/profile/arbys_163389...,"http://www.arbys.com,https://locations.arbys.c..."
4,AWQ6MUvo3-Khe5l_j3SG,2018-06-26T02:59:43Z,2018-06-26T02:59:43Z,2118 Mt Zion Parkway,Fast Food Restaurant,Morrow,US,us/ga/morrow/2118mtzionparkway/1305117222,33.562738,-84.321143,Steak 'n Shake,30260.0,GA,https://foursquare.com/v/steak-n-shake/4bcf77a...,http://www.steaknshake.com/locations/23851-ste...


# Data Transformation

In [4]:
# get only the columns we need
income_new = income[['City','State_ab', 'Mean', 'Median']]
# Rename columns
income_complete = income_new.rename(columns={'State_ab':'State', 'Mean':'Income_Mean', 'Median':'Income_Median'})
# group income by city
income_mean = income_complete.groupby(["City","State"]).mean()['Income_Mean']
income_median = income_complete.groupby(["City","State"]).median()['Income_Median']
income_by_city = pd.DataFrame({ "Income_Mean":income_mean, "Income_Median":income_median })
# convert city into a column
income_by_city = income_by_city.reset_index(level=0)
income_by_city = income_by_city.reset_index(level=0)
income_by_city.head()


,State,City,Income_Mean,Income_Median
0,AL,Abbeville,40518.000000,25216.000000
1,LA,Abbeville,30647.000000,25628.333333
2,SC,Abbeville,45585.666667,32245.000000
3,WI,Abbotsford,58254.000000,44919.000000
4,MD,Aberdeen,96480.000000,80370.000000


In [9]:
len(income_by_city)

11228

In [5]:
# Filtering only cities in the US
fast_food_us = fast_food.loc[fast_food['country'] == 'US']
fast_food_us['country'].unique()

array(['US'], dtype=object)

In [6]:
# drop duplicates
fast_food_us = fast_food_us.drop_duplicates(subset=['address'])
# get only the columns we need
fast_food_us_new = fast_food_us[['address','city', 'province','name','postalCode','categories']]
# Rename province to State, postalCode to ZipCode, and title-case others
fast_food_complete = fast_food_us_new.rename(columns={'address':'Address','city':'City','province': 'State', 'name':'Name','categories':'Categories','postalCode':'Zip_Code'})
fast_food_complete.head()

,Address,City,State,Name,Zip_Code,Categories
0,800 N Canal Blvd,Thibodaux,LA,SONIC Drive In,70301.0,American Restaurant and Fast Food Restaurant
2,206 Wears Valley Rd,Pigeon Forge,TN,Taco Bell,37863.0,Fast Food Restaurant
3,3652 Parkway,Pigeon Forge,TN,Arby's,37863.0,Fast Food
4,2118 Mt Zion Parkway,Morrow,GA,Steak 'n Shake,30260.0,Fast Food Restaurant
5,9768 Grand River Ave,Detroit,MI,Wendy's,48204.0,Fast Food Restaurant


## Connect to Postgres and Convert the DataFrames into DataBase

In [7]:
protocol = 'postgresql'
# Change username and password
username = ''
password = ''
host = 'localhost'
port = 5432
database_name = ''
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [10]:
fast_food_complete.to_sql(name='fast_food', con=engine, if_exists='replace', index=False)
income_by_city.to_sql(name='income', con=engine, if_exists='replace', index=False)

228